# Notebook For Compressing Healthcare Data

In [1]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StructType, StructField
import numpy as np
from rapidfuzz import fuzz
import sqlite3

# Ensure JAVA_HOME is set
#os.environ['JAVA_HOME'] = 'C:\Program Files\Java'  # Update this path to your Java installation

# Ensure SPARK_HOME is set
os.environ['SPARK_HOME'] = 'C:/Users/John/AppData/Local/Programs/Python/Python312/Lib/site-packages/pyspark'  # Update this path to your Spark installation

# Ensure HADOOP_HOME is set
#os.environ['HADOOP_HOME'] = 'C:/Hadoop'  # Update this path to your Hadoop installation

# Initialize findspark
findspark.init()

In [2]:
# Initialize SparkSession
spark = SparkSession.builder.master("local[*]") \
    .enableHiveSupport() \
    .config("spark.driver.memory", "4g") \
    .appName("Compress Healthcare Data") \
    .getOrCreate()

In [ ]:
############## Helper Functions ################

def read_all_files_in_data_dir(directory):
    """
    Read in all parquet datasets in a directory and read them into a pyspark dataframe
    
    :return dict -> key:(filename): value (pyspark_data storing data)
    """
    dataframes_list = {}
    for entry in os.scandir(directory):  
        if entry.is_file() and entry.path.endswith('.parquet'):  # check if it's a parquet file
            file_path = entry.path
            # Read File into pyspark dataframe
            name_of_file = entry.path.split('/')[-1].split('.')[0]
            print(f"Reading File Path: {file_path}")
            df = spark.read.parquet(file_path)
            dataframes_list[name_of_file] = df
    print("SUCCESS: All Parquet Data in Data Directory Read")
    return dataframes_list

def print_schema_for_all_dataframes(dataframes_list):
    """
    Print Schema For All Dataframes in dataframe list and write out to file
    """
    with open('all_data_schemas.txt', 'w') as file:
        for df_key in dataframes_list.keys():
            df = dataframes_list[df_key]
            schema_string = df._jdf.schema().treeString()
            file.write(f"Schema for file: {df_key}\n")
            file.write(schema_string)
            file.write('\n----------------------------\n')

def convert_add_pyspark_df_to_pandas(dataframes_list):
    """
    Take list of pyspark dataframes, convert to pandas dataframes, convert to pickle file, and write to pandas_dataframes directory
    """
    for df_key in dataframes_list.keys():
        dataframes_list[df_key].write.mode('overwrite').parquet(f'parquet_files/{df_key}')
        # pandas_df = dataframes_list[df_key].toPandas()
        # pandas_df.to_pickle(f'pandas_dataframes/{df_key}.pkl')
    print("SUCCESS: All dataframes have been written as parquet files")


def load_dataframes_to_sql(dataframes_list, db_name='healthcare_data.db'):
    """
    Load all dataframes into a SQL database file.
    
    :param dataframes_list: dict -> key:(filename): value (pyspark_data storing data)
    :param db_name: str -> name of the database file
    """
    try:
        # Create a connection to the SQLite database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        print(f"Connected to database {db_name}")
        
        for df_key in dataframes_list.keys():
            # Convert PySpark DataFrame to Pandas DataFrame
            pandas_df = dataframes_list[df_key].toPandas()
            
            # Load the DataFrame into the SQL database
            pandas_df.to_sql(df_key, conn, if_exists='replace', index=False)
            print(f"SUCCESS: {df_key} loaded into {db_name}")
        
        # Close the connection
        conn.close()
        print("SUCCESS: All dataframes have been loaded into the SQL database")
    
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"Error: {e}")

def load_dataframes_to_sql_in_chunks(dataframes_list, db_name='healthcare_data.db', chunk_size=10000):
    """
    Load all dataframes into a SQL database file in chunks to handle RAM constraints.
    
    :param dataframes_list: dict -> key:(filename): value (pyspark_data storing data)
    :param db_name: str -> name of the database file
    :param chunk_size: int -> number of rows per chunk
    """
    try:
        # Create a connection to the SQLite database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        print(f"Connected to database {db_name}")
        
        for df_key in dataframes_list.keys():
            # Convert PySpark DataFrame to Pandas DataFrame in chunks
            df = dataframes_list[df_key]
            total_rows = df.count()
            num_chunks = total_rows // chunk_size + 1
            
            for i in range(num_chunks):
                pandas_df = df.limit(chunk_size).offset(i * chunk_size).toPandas()
                pandas_df.to_sql(df_key, conn, if_exists='append', index=False)
                print(f"SUCCESS: Chunk {i+1}/{num_chunks} of {df_key} loaded into {db_name}")
        
        # Close the connection
        conn.close()
        print("SUCCESS: All dataframes have been loaded into the SQL database in chunks")
    
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"Error: {e}")


In [4]:
dataframes_list = read_all_files_in_data_dir('../cleaned_data')

Reading File Path: ../cleaned_data\diagnoses_icd.parquet
Reading File Path: ../cleaned_data\discharge.parquet
Reading File Path: ../cleaned_data\discharge_detail.parquet
Reading File Path: ../cleaned_data\drgcodes.parquet
Reading File Path: ../cleaned_data\d_hcpcs.parquet
Reading File Path: ../cleaned_data\d_icd_diagnoses.parquet
Reading File Path: ../cleaned_data\d_icd_procedures.parquet
Reading File Path: ../cleaned_data\d_labitems.parquet
Reading File Path: ../cleaned_data\emar.parquet
Reading File Path: ../cleaned_data\emar_detail.parquet
Reading File Path: ../cleaned_data\hcpcsevents.parquet
Reading File Path: ../cleaned_data\microbiologyevents.parquet
Reading File Path: ../cleaned_data\omr.parquet
Reading File Path: ../cleaned_data\patients.parquet
Reading File Path: ../cleaned_data\pharmacy.parquet
Reading File Path: ../cleaned_data\prescriptions.parquet
Reading File Path: ../cleaned_data\procedures_icd.parquet
Reading File Path: ../cleaned_data\radiology.parquet
Reading File Pa

In [5]:
print_schema_for_all_dataframes(dataframes_list)

In [9]:
load_dataframes_to_sql_in_chunks(dataframes_list)

Connected to database healthcare_data.db
SUCCESS: Chunk 1/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 2/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 3/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 4/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 5/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 6/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 7/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 8/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 9/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 10/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 11/637 of cleaned_data\diagnoses_icd loaded into healthcare_data.db
SUCCESS: Chunk 12/637 of cleaned_data\diagnoses_icd loa

In [ ]:
#convert_add_pyspark_df_to_pandas(dataframes_list)

In [ ]:
spark.stop()

In [ ]:
# Example on how to read the data

# import pandas as pd
# import pyarrow

# df = pd.read_parquet('parquet_files/d_hcpcs/')
# df.head()